In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [1]:
with open('census.txt') as f:
    census_api_key = f.readline().split('\n')[0]

# Gathering Data
 - Using census api and brfss

## ACS Census Data

In [2]:
#required packages
import pandas as pd;
# !pip install census;
# !pip install us;

In [3]:
from census import Census
from us import states
c = Census(census_api_key)
#pull % of people in each county with insurance
varnames = {
    'NAME':'county_name',
    'B992701_001E':'total_for_insurance',
    'B992701_002E':'some_insurance',
    'B992701_003E':'no_insurance',
    'B21004_001E':'median_income',
    'C24070_011E':'has_edu_health_socservices_job',
    'C24070_001E':'total_for_job_industry'
}
variables_to_get = tuple( list( varnames.keys()))

years = [2012,2013,2014,2015,2016]
dflist = []
for yr in years:
    dflist.append(
        pd.DataFrame(
            c.acs5.get(variables_to_get, geo={'for': 'county:*', 'in': 'state:{}'.format(states.CA.fips)},year=yr)
        ))
 
# dflist[0] = 2012 dataset
# dflist[4] = 2016 dataset
for df in dflist:
    df.rename(columns=varnames,inplace=True)
    #put into fractions
    df.loc[:,'some_insurance'] = df.loc[:,'some_insurance'] / df.loc[:,'total_for_insurance']
    df.loc[:,'no_insurance'] = df.loc[:,'no_insurance'] / df.loc[:,'total_for_insurance']
    df.loc[:,'has_edu_health_socservices_job'] = df.loc[:,'has_edu_health_socservices_job'] / df.loc[:,'total_for_job_industry']
    df.rename(columns=varnames,inplace=True)
    df.set_index('county',inplace=True)
    df.drop(columns=['state','total_for_insurance','total_for_job_industry'],inplace=True)
    
    display( df.head(3) )
    display( df.shape )

,median_income,some_insurance,no_insurance,has_edu_health_socservices_job,county_name
county,,,,,
001,34223.0,0.084999,0.915001,0.226367,"Alameda County, California"
003,27464.0,0.092732,0.907268,0.241697,"Alpine County, California"
005,25559.0,0.102707,0.897293,0.200112,"Amador County, California"


(58, 5)

,median_income,some_insurance,no_insurance,has_edu_health_socservices_job,county_name
county,,,,,
001,33894.0,0.089769,0.910231,0.227650,"Alameda County, California"
003,26357.0,0.095279,0.904721,0.175589,"Alpine County, California"
005,26176.0,0.110027,0.889973,0.194385,"Amador County, California"


(58, 5)

,median_income,some_insurance,no_insurance,has_edu_health_socservices_job,county_name
county,,,,,
001,34086.0,0.100290,0.899710,0.225663,"Alameda County, California"
003,25278.0,0.147255,0.852745,0.134653,"Alpine County, California"
005,25097.0,0.121449,0.878551,0.184728,"Amador County, California"


(58, 5)

,median_income,some_insurance,no_insurance,has_edu_health_socservices_job,county_name
county,,,,,
001,34761.0,0.111247,0.888753,0.224455,"Alameda County, California"
003,19420.0,0.142352,0.857648,0.154412,"Alpine County, California"
005,26420.0,0.128893,0.871107,0.194385,"Amador County, California"


(58, 5)

,median_income,some_insurance,no_insurance,has_edu_health_socservices_job,county_name
county,,,,,
001,36303.0,0.133606,0.866394,0.221210,"Alameda County, California"
003,22500.0,0.184966,0.815034,0.181598,"Alpine County, California"
005,27873.0,0.133436,0.866564,0.190884,"Amador County, California"


(58, 5)

## CDC Ground Truth for Labels


In [4]:
# !pip install xlrd
healthcare_df = pd.read_excel('hcp-flu-vax-by-county-20162017data.xlsx')

In [5]:
healthcare_df.head()
healthcare_df.loc[:,['County','All Health Care Personnel (HCP) Percent Vaccinated']].set_index('County').rename(columns={'All Health Care Personnel (HCP) Percent Vaccinated':'hcp_vaccinated'})

,hcp_vaccinated
County,
Alameda,87.0
Amador,86.0
Butte,77.1
Calaveras,97.6
Contra Costa,82.5
Del Norte,89.1
El Dorado,89.0
Fresno,86.9
Glenn,84.0


In [6]:
healthcare_df.shape

(55, 14)

In [7]:
dfbrfss = pd.read_sas('LLCP2016.XPT')

In [8]:
dfbrfss.head()
dfbrfss.shape

# _STATE = state fips code, california=6
# MEDCOST = Could Not See Doctor Because of Cost
# SEX
# EDUCA = Education Level
# EMPLOY1 = Employment Status
# INCOME2 = Income Level
# SEATBELT

#2016 BRFSS did the influenza cluster in CA, MS, MO, NE, CT, MA, TX
# state_labels = ['CA','MS','MO','NE','CT','MA','TX']
# state_pops = [39536653, 2984100, 6044171, 1868516, 3588184, 6859819, 26448193]     



# ca_brfss = dfbrfss.loc[ dfbrfss['_STATE']==6 ]
# ms_brfss = dfbrfss.loc[ dfbrfss['_STATE']==28 ]
# mo_brfss = dfbrfss.loc[ dfbrfss['_STATE']==29 ]
# ne_brfss = dfbrfss.loc[ dfbrfss['_STATE']==31 ]
# ct_brfss = dfbrfss.loc[ dfbrfss['_STATE']==9 ]
# ma_brfss = dfbrfss.loc[ dfbrfss['_STATE']==25 ]
# tx_brfss = dfbrfss.loc[ dfbrfss['_STATE']==48 ]



(486303, 275)

In [9]:
states.STATES

[<State:Alabama>,
 <State:Alaska>,
 <State:Arizona>,
 <State:Arkansas>,
 <State:California>,
 <State:Colorado>,
 <State:Connecticut>,
 <State:Delaware>,
 <State:District of Columbia>,
 <State:Florida>,
 <State:Georgia>,
 <State:Hawaii>,
 <State:Idaho>,
 <State:Illinois>,
 <State:Indiana>,
 <State:Iowa>,
 <State:Kansas>,
 <State:Kentucky>,
 <State:Louisiana>,
 <State:Maine>,
 <State:Maryland>,
 <State:Massachusetts>,
 <State:Michigan>,
 <State:Minnesota>,
 <State:Mississippi>,
 <State:Missouri>,
 <State:Montana>,
 <State:Nebraska>,
 <State:Nevada>,
 <State:New Hampshire>,
 <State:New Jersey>,
 <State:New Mexico>,
 <State:New York>,
 <State:North Carolina>,
 <State:North Dakota>,
 <State:Ohio>,
 <State:Oklahoma>,
 <State:Oregon>,
 <State:Pennsylvania>,
 <State:Rhode Island>,
 <State:South Carolina>,
 <State:South Dakota>,
 <State:Tennessee>,
 <State:Texas>,
 <State:Utah>,
 <State:Vermont>,
 <State:Virginia>,
 <State:Washington>,
 <State:West Virginia>,
 <State:Wisconsin>,
 <State:Wyoming

In [10]:
from census import Census
from us import states
state_list = states.STATES
flu_brfss_list=[]

for state in state_list:
    flu_brfss_list.append( dfbrfss.loc[ dfbrfss['_STATE']== int(state.fips) ] )

In [11]:
for state_brfss in flu_brfss_list:
    display( state_brfss.shape )

(7031, 275)

(2914, 275)

(10952, 275)

(5298, 275)

(11393, 275)

...

(9002, 275)

(14259, 275)

(7151, 275)

(5271, 275)

(4497, 275)

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
feature_list = ['MEDCOST','SEX','_RACE','EDUCA','INCOME2','HLTHPLN1']

state_data=[]
for i, state_brfss in enumerate( flu_brfss_list ):
    state_data.append(state_brfss.loc[:,
    ('MEDCOST','SEX','_RACE','EDUCA','INCOME2','HLTHPLN1','FLUSHOT6','_LLCPWT')])

    #only keeping Yes and No, which is around 99% of the responses
    state_data[i] = state_data[i].loc[ (state_data[i]['FLUSHOT6']==1) | (state_data[i]['FLUSHOT6']==2) ]

    #only keeping valid income answers 1-8
    state_data[i] = state_data[i].loc[ (state_data[i]['INCOME2']<=8) ]
    state_data[i] = state_data[i].loc[ (state_data[i]['_RACE']<=8) ]
    state_data[i] = state_data[i].loc[ (state_data[i]['EDUCA']<=6) ]
    state_data[i] = state_data[i].loc[ (state_data[i]['SEX']<=2) ]
    state_data[i] = state_data[i].loc[ (state_data[i]['MEDCOST']<=2) ]
    state_data[i] = state_data[i].loc[ (state_data[i]['HLTHPLN1']<=2) ]
    state_data[i] = state_data[i].astype(str)
    state_data[i]['INCOME2'] = state_data[i]['INCOME2'].astype(float)
    state_data[i]['EDUCA'] = state_data[i]['EDUCA'].astype(float)

In [ ]:
#sampling 10,000 people from each state using weights
state_data_samples=[]
# for (data, pop) in zip(state_data, state_pops):
    
df = state_data[4].sample( n=10000, weights=state_data[4]['_LLCPWT'], replace=True)
state_data_samples.append(df)

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X_list = []
y_list = []
X_train_list = []
X_test_list  = []
y_train_list = []
y_test_list  = []
for i, data in enumerate(state_data_samples):
    # Features
    X = pd.get_dummies(data[feature_list],drop_first=True)
    # Labels
    y = data['FLUSHOT6']
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test
    X_list.append(X)
    y_list.append(y)
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

for (X,y,state) in zip(X_list, y_list,state_list):
    print(state.name)
    features = X
    labels = y
    plt.figure(figsize=(8,8))
    sns.heatmap(features.corr(), annot=True, square=True, cmap='coolwarm')
    savefig('feature_correlation_'+state.name+'.png')
    plt.pause(0.05)
   
plt.show()

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
#Train the model using the training sets y_pred=clf.predict(X_test)
y_pred_list=[]
clf_list=[]
for (X_train,y_train, X_test) in zip(X_train_list, y_train_list,X_test_list):
    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    y_pred_list.append( clf.predict(X_test) )
    clf_list.append(clf)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
for (y_test,y_pred,state_abbrev) in zip(y_test_list,y_pred_list,state_labels):
    print("Accuracy for "+state_abbrev+": ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
feature_imp_list=[]
for (clf,X,state_abbrev) in zip(clf_list,X_list,state_labels):
    feature_imp = pd.Series(clf.feature_importances_,index=X.columns.tolist()).sort_values(ascending=False)
    feature_imp_list.append(feature_imp)

In [ ]:
for (feature_imp,state_abbrev) in zip(feature_imp_list,state_labels):
    plt.figure(figsize=(5,3)) 
    # Creating a bar plot
    sns.barplot(x=feature_imp, y=feature_imp.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score for'+state_abbrev)
    plt.ylabel('Features')
    plt.title("Visualizing Important Features for"+state_abbrev)
    plt.legend()
plt.show()

In [ ]:
!pip install graphviz
!pip install pydotplus
!apt-get install python-pydot -y --allow-unauthenticated

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

for clf in clf_list:
    estimator = clf.estimators_[12]
    dot_data = StringIO()
    export_graphviz(estimator, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    Image(graph.create_png())

```
_RACE
1 White only, non-Hispanic
2 Black only, non-Hispanic
3 American Indian or Alaskan Native only,Non-Hispanic
4 Asian only, non-Hispanic
5 Native Hawaiian or other Pacific Islander only,Non-Hispanic
6 Other race only, non-Hispanic
7 Multiracial, non-Hispanic
8 Hispanic


EDUCA
# 1=Never attended school or only kindergarten
2=Grades 1 through 8 (Elementary)
3=Grades 9 through 11 (Some high school)
4=Grade 12 or GED (High school graduate) 
5=College 1 year to 3 years (Some college or technical school)
6=College 4 years or more (College graduate) 
9=refused


 INCOME2
1 Less than $10,000
2 Less than $15,000 ($10,000 to less than $15,000)
3 Less than $20,000 ($15,000 to less than $20,000)
4 Less than $25,000 ($20,000 to less than $25,000)
5 Less than $35,000 ($25,000 to less than $35,000)
6 Less than $50,000 ($35,000 to less than $50,000)
7 Less than $75,000 ($50,000 to less than $75,000)
8 $75,000 or more 127,081 26.34 27.47
77 Don’t know/Not sure 35,338 7.32 8.24
99 Refused 42,177 8.74 7.85
```
